## SYCL Kernel for 1 index (1D)

In [ ]:
%%writefile src/functionED.xx
#include <sycl/sycl.hpp>

void Field::evalDensity() {
    
    sycl::queue q(sycl::default_selector_v);
    std::cout << " Running on "
            << q.get_device().get_info<sycl::info::device::name>() << std::endl;
    
    vector<float> field;
    int natm = wf.natm;
    const int npri = wf.npri;
    int norb = wf.norb;
    int npy = npoints_y;
    int npz = npoints_z;
    float x0 = xmin;
    float y0 = ymin;
    float z0 = zmin;
    float hp = delta;
    float *field_local = new float[nsize];

    std::cout << " Points ( " << npoints_x << "," << npoints_y << "," << npoints_z
            << ")" << std::endl;
    std::cout << " TotalPoints : " << npoints_x * npoints_y * npoints_z
            << std::endl;

    float *coor = new float[3 * natm];
    for (int i = 0; i < natm; i++) {
        Rvector R(wf.atoms[i].getCoors());
        coor[3 * i] = R.get_x();
        coor[3 * i + 1] = R.get_y();
        coor[3 * i + 2] = R.get_z();
    }
  // Here we start the sycl kernel
    {
        
        auto field_buff = sycl::buffer{field_local, sycl::range<1>(nsize)};
        auto icnt_buff = sycl::buffer{wf.icntrs.data(), sycl::range<1>(npri)};
        auto vang_buff = sycl::buffer{wf.vang.data(), sycl::range<1>(3*npri)};
        auto coor_buff = sycl::buffer{coor, sycl::range<1>(3*wf.natm)};
        auto eprim_buff = sycl::buffer{wf.depris.data(), sycl::range<1>(npri)};
        auto coef_buff = sycl::buffer{wf.dcoefs.data(), sycl::range<1>(npri * norb)};
        auto nocc_buff = sycl::buffer{wf.dnoccs.data(), sycl::range<1>(norb)};



        q.submit([&](sycl::handler &h) {

            sycl::accessor field_acc { field_buff, h, sycl::write_only};
            sycl::accessor icnt_acc  { icnt_buff,  h, sycl::read_only};
            sycl::accessor vang_acc  { vang_buff,  h, sycl::read_only};
            sycl::accessor coor_acc  { coor_buff,  h, sycl::read_only};
            sycl::accessor eprim_acc  { eprim_buff, h, sycl::read_only};
            sycl::accessor coef_acc  { coef_buff,  h, sycl::read_only};
            sycl::accessor nocc_acc  { nocc_buff,  h, sycl::read_only};


// 1D index
            h.parallel_for(sycl::range<1>(nsize), [=](sycl::id<1> idx) {
                float cart[3];
                const int k = (int)idx % npz;
                const int j = ((int)idx / npz) % npy;
                const int i = (int)idx / (npz * npy);
                
                cart[0] = x0 + i * hp;
                cart[1] = y0 + j * hp;
                cart[2] = z0 + k * hp;
                
                const int *icnt_ptr = icnt_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                const int *vang_ptr = vang_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                const float *coor_ptr = coor_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                const float *eprim_ptr = eprim_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                const float *nocc_ptr = nocc_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                const float *coef_ptr = coef_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                
                field_acc[idx] = Density(norb, npri, icnt_ptr, vang_ptr, cart, coor_ptr, eprim_ptr, nocc_ptr, coef_ptr);
            });
        });
        q.wait();
      }
  // End the kernel of SYCL

    for (int i = 0; i < nsize; i++)
        field.push_back(field_local[i]);
#ifdef PRINTCUBE
    dumpCube(xmin, ymin, zmin, delta, npoints_x, npoints_y, npoints_z, field,
           "densitySYCL1.cube");
    //dumpXYZ("structure.xyz");
#endif
    
    delete[] coor;
    delete[] field_local;
}

All rights reserved. Copyright Argonne National Laboratory UChicago LLC.